In [13]:
# Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [14]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\bluec\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [15]:
# URL of page to be scraped
# url = 'http://house.speakingsame.com/p.php?q=Perth%2C+WA'
url = 'http://house.speakingsame.com/p.php?q=West+Perth%2C+WA'
browser.visit(url)

In [16]:
addresses = []
properties_info = []

pages = np.arange(0, 30, 1)

for page in pages:
    
#     page_url = "http://house.speakingsame.com/p.php?q=Perth&p="+ str(page) +"&s=1&st=&type=&count=300&region=Perth&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    page_url = "http://house.speakingsame.com/p.php?q=northbridge&p="+ str(page) +"&s=1&st=&type=&count=300&region=northbridge&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    
    browser.visit(page_url)
   
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    addr_spans = soup.find_all('span', class_='addr')

    for span in addr_spans:
        addresses.append(span.a.text)
    addresses

    tables = soup.find_all('table', style="font-size:13px") #tables list (except 0,1 index) contians list of properties in the page

    for i in range(2,12):
        try:
            info = []
            trs = tables[i].find_all('tr')
            for tr in trs:
                info.append(tr.td.text)
        except:
            info.append('')
        properties_info.append(info)
        
  
    
properties_info  

[['Sold $440,000 in Jul 2021',
  'Last Sold $382,000 in Oct 2018',
  'Rent $475pw in Dec 2017',
  'Apartment: 2  1  1 ',
  'Land size: 108 sqm | Building size: 84 sqm',
  'Agent: Claude Iaconi',
  'UNDER OFFER!!. 6/11 Shenton Street, Northbridge Taking in all northbridge has to offer this lovely apartment is the perfect place to call home or simply a welcome addition to any real estate...more',
  'Floorplan | Street view | Nearby'],
 ['Sold $320,000 in May 2021',
  'Rent $300pw in Sep 2016',
  'Apartment: 1  1  1 ',
  'Land size: 627 sqm | Building size: 56 sqm',
  'Under Offer... Just Listed - Super Convenient Location - Great Value !! Dont miss this one - Boutique complex 12 apartments. Very low maintenance lock and leave lifestyle. Short stroll to Cafes &...more',
  'Street view | Nearby'],
 ['Sold $379,000 in May 2021',
  'Rent $380pw in Aug 2019',
  'Apartment: 2  1  1 ',
  'Agent: Brendon Habak',
  'UNDER OFFER. Designed by renowned architects and developer, Baltinas MADE, Vélo i

In [5]:
len(properties_info)

300

In [6]:
len(addresses)

300

In [7]:
Properties = []
for i in range(0, len(addresses)):
    
    Property_dict = {}
    Property_dict['address'] = addresses[i]
    
    for j in range (0, len(properties_info[i])):
        
        try:
                    
            if properties_info[i][j].split(' ')[0] == "Sold": 
                Property_dict['price'] = properties_info[i][j].split(' ')[1]
                Property_dict['sold date'] = properties_info[i][j].split('in ')[1]

            if properties_info[i][j].split(' ')[0] in ["Apartment:", "House:", "Townhouse:"]:
                Property_dict['type'] = properties_info[i][j].split(' ')[0].split(':')[0]
                Property_dict['bedrooms'] = properties_info[i][j].split(' ')[1]
                try:
                    Property_dict['bathrooms'] = properties_info[i][j].split(' ')[3]
                except:
                    Property_dict['bathrooms'] = ''
                try:
                    Property_dict['car space'] = properties_info[i][j].split(' ')[5]
                except:
                    Property_dict['car space'] = ''
                    
            if properties_info[i][j].split(' ')[0] == "Agent:":
                Property_dict['agent'] = properties_info[i][j].split(': ')[1]
                
            if properties_info[i][j].split(' ')[0] == "Rent":
                Property_dict['rent'] = properties_info[i][j].split(' ')[1]
                Property_dict['rent date'] = properties_info[i][j].split('in ')[1]
                
            if properties_info[i][j].split(' ')[1] == "size:":
                if properties_info[i][j].split(' ')[0] == "Land":
                    Property_dict['Land size'] = properties_info[i][j].split(' ')[2]
                    try:
                        if properties_info[i][j].split(' ')[5] == "Building":
                            Property_dict['Building size'] = properties_info[i][j].split(' ')[7]
                    except:
                        continue
                elif properties_info[i][j].split(' ')[0] == "Building":
                    Property_dict['Building size'] = properties_info[i][j].split(' ')[2]
            
            
        except:
            continue

  
    Properties.append(Property_dict)
        
Properties

[{'address': '10/2 Mayfair Street',
  'price': '$410,000',
  'sold date': 'Aug 2021',
  'rent': '$400pw',
  'rent date': 'Jun 2019',
  'type': 'Apartment',
  'bedrooms': '2',
  'bathrooms': '2',
  'car space': '1',
  'Land size': '102',
  'Building size': '81'},
 {'address': '42/33 MALCOLM STREET',
  'price': '$1,050,000',
  'sold date': 'Jul 2021',
  'rent': '$1,200pw',
  'rent date': 'Feb 2019',
  'type': 'House',
  'bedrooms': '3',
  'bathrooms': '2',
  'car space': '2',
  'Land size': '1,701',
  'Building size': '140'},
 {'address': '6/16 Carr Street',
  'price': '$550,000',
  'sold date': 'Jul 2021',
  'type': 'Townhouse',
  'bedrooms': '3',
  'bathrooms': '2',
  'car space': '2',
  'Land size': '190',
  'agent': 'Ashby Farrell'},
 {'address': '11/875 Wellington Street',
  'price': '$110,000',
  'sold date': 'Jul 2021',
  'type': 'Apartment',
  'bedrooms': '',
  'bathrooms': '',
  'car space': '',
  'agent': 'Daniel Porcaro'},
 {'address': '6/1 Carr Street',
  'price': '$442,000',

In [8]:
len(Properties)

300

In [10]:
Northbridge_df = pd.DataFrame(Properties)

In [17]:
Northbridge_df

,address,price,sold date,rent,rent date,type,bedrooms,bathrooms,car space,Land size,Building size,agent
0,10/2 Mayfair Street,"$410,000",Aug 2021,$400pw,Jun 2019,Apartment,2,2,1,102,81,NaN
1,42/33 MALCOLM STREET,"$1,050,000",Jul 2021,"$1,200pw",Feb 2019,House,3,2,2,"1,701",140,NaN
2,6/16 Carr Street,"$550,000",Jul 2021,NaN,NaN,Townhouse,3,2,2,190,NaN,Ashby Farrell
3,11/875 Wellington Street,"$110,000",Jul 2021,NaN,NaN,Apartment,,,,NaN,NaN,Daniel Porcaro
4,6/1 Carr Street,"$442,000",Jul 2021,$575pw,Mar 2021,Apartment,3,1,1,156,88,Annette O'Brien-Oxley
...,...,...,...,...,...,...,...,...,...,...,...,...
295,18/161 Charles Street,"$330,000",Sep 2018,NaN,NaN,Apartment,2,1,1,"2,233",NaN,ACTON Victoria Park
296,17/33 Malcolm Street,"$606,000",Aug 2018,$520pw,Sep 2017,NaN,NaN,NaN,NaN,120,76,Caporn Young Estate Agents - Claremont
297,69/9 Delhi Street,"$635,000",Aug 2018,$575pw,Jul 2016,House,3,2,2,154,101,Mack Hall Real Estate - Claremont
298,3/28 Carr Street,"$740,000",Aug 2018,NaN,NaN,Townhouse,3,2,2,169,162,Mack Hall Real Estate - Claremont


In [18]:
Northbridge_df.to_csv("Northbridge_properties.csv")